In [1]:
pip install tensorflow_datasets

Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import numpy as np # linear algebra
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd #data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import PIL
import PIL.Image
from tensorflow import keras
import tensorflow_datasets as tfds

## A )

Đoạn code tiếp theo thực hiện việc khởi dựng mô hình CNN với kiến trúc để phân loại cho 11 lớp khuôn mặt

In [8]:
# Trains a model to classify images of 11 classes
def gen_model():
    # Defines & compiles the model
    model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150,
    3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    keras.layers.Dropout(rate=0.15), #adding dropout regularization throughout the model to deal with overfitting
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    keras.layers.Dropout(rate=0.1),

    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    keras.layers.Dropout(rate=0.10),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # 3 output neuron for the 3 classes of Animal Images
    tf.keras.layers.Dense(11, activation='softmax')
    ])
    
    from tensorflow.keras.optimizers import RMSprop
    
    model.compile(loss='categorical_crossentropy',
    optimizer="adam",
    metrics=['acc'])
    return model

Tiếp theo chúng ta thiết lập dữ liệu và gọi mô hình để huấn luyện và dự báo

In [9]:
# Trains a model to classify images of 3 classes: cat, dog, and panda
def train_test_animals():
    # Creates an instance of an ImageDataGenerator called train_datagen, and a train_generator, train_datagen.flow_from_directory
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    #splits data into training and testing(validation) sets
    train_datagen =ImageDataGenerator(rescale=1./255, validation_split=0.2)
    import matplotlib.pyplot as plt

    #training data
    train_generator = train_datagen.flow_from_directory('/face_data/face',
    # Source directory
    target_size=(150, 150), # Resizes images
    batch_size=15,
    class_mode='categorical',subset = 'training')
    epochs = 1
    #Testing data
    validation_generator = train_datagen.flow_from_directory('/face_data/validation',
    target_size=(150, 150),
    batch_size=5,
    class_mode='categorical',
    subset='validation') # set as validation data
    model = gen_model()
    #Model fitting for a number of epochs
    history = model.fit_generator(
    train_generator,
    steps_per_epoch=150,
    epochs=epochs,
    validation_data = validation_generator,

    validation_steps = 50,
    verbose=1)

    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    #This code is used to plot the training and validation accuracy
    epochs_range = range(1, epochs + 1)  # Thay đổi đoạn này
    
    model.save_weights('/saved_weights.h5')
    # returns accuracy of training
    from sklearn.metrics import accuracy_score, precision_score, recall_score

    # Dự đoán cho tập train
    train_predictions = model.predict(train_generator)
    train_labels = train_generator.classes
    train_accuracy = accuracy_score(train_labels, train_predictions.argmax(axis=1))
    train_precision = precision_score(train_labels, train_predictions.argmax(axis=1), average='weighted')
    train_recall = recall_score(train_labels, train_predictions.argmax(axis=1), average='weighted')

    # Dự đoán cho tập test
    test_predictions = model.predict(validation_generator)
    test_labels = validation_generator.classes
    test_accuracy = accuracy_score(test_labels, test_predictions.argmax(axis=1))
    test_precision = precision_score(test_labels, test_predictions.argmax(axis=1), average='weighted')
    test_recall = recall_score(test_labels, test_predictions.argmax(axis=1), average='weighted')

    print("Train Accuracy:", train_accuracy)
    print("Train Precision:", train_precision)
    print("Train Recall:", train_recall)

    print("Test Accuracy:", test_accuracy)
    print("Test Precision:", test_precision)
    print("Test Recall:", test_recall)


## B )

In [10]:
train_test_animals()

Found 132 images belonging to 11 classes.
Found 33 images belonging to 11 classes.


C:\Users\FPT\AppData\Local\Temp\ipykernel_4936\1176710479.py:24: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


5/7 [====================>.........] - ETA: 0s

c:\Users\FPT\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


7/7 [==============================] - 0s 15ms/step
Train Accuracy: 0.09090909090909091
Train Precision: 0.008391608391608392
Train Recall: 0.09090909090909091
Test Accuracy: 0.09090909090909091
Test Precision: 0.00879765395894428
Test Recall: 0.09090909090909091


c:\Users\FPT\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Do dữ liệu đầu vào ít nên được sử dụng hết trong quá trình huấn luyện mô hình. Khiến cho  quá trình huấn luyện đã bị gián đoạn và đưa ra kết quả với độ chính xác thâp

## C )

Kết hợp giảm chiều và sử dụng mô hình ANN

In [11]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

train_dir = '/face_data/face'
validation_dir = '/face_data/validation'

# Function to resize images
def resize_image(image_path, target_size=(150, 150)):
    image = Image.open(image_path)
    image = image.resize(target_size).convert("RGB")
    return np.array(image)

def load_data(train_dir):
    images = []
    labels = []
    for label in os.listdir(train_dir):
        
        label_dir = os.path.join(train_dir, label)
        for file in os.listdir(label_dir):
            image_path = os.path.join(label_dir, file)
            # Resize image to 150x150
            try:
                resized_image = resize_image(image_path)
                images.append(resized_image)
                # Assign label based on folder name
                if label == 'centerlight':
                    labels.append(0)  
                elif label == 'glasses':
                    labels.append(1)
                elif label == 'happy':
                    labels.append(2)
                elif label == 'leftlight':
                    labels.append(3)  
                elif label == 'noglasses':
                    labels.append(4)
                elif label == 'normal':
                    labels.append(5)
                elif label == 'rightlight':
                    labels.append(6)
                elif label == 'sad':
                    labels.append(7)  
                elif label == 'sleepy':
                    labels.append(8)   
                elif label == 'surprise':
                    labels.append(9)
                elif label == 'wink':
                    labels.append(10)                                                           
            except Exception as e:
                print("Error processing image:", image_path)
                print("Error message:", str(e))
    return np.array(images), np.array(labels)


# Load and prepare data
images, labels = load_data(train_dir)


#Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Flatten the images
X_train_flatten = X_train.reshape(X_train.shape[0], -1)
X_test_flatten = X_test.reshape(X_test.shape[0], -1)

# Train a Softmax Regression model
model = LogisticRegression(max_iter=1000, multi_class='multinomial')
model.fit(X_train_flatten, y_train)

# Evaluate the model
y_pred = model.predict(X_test_flatten)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.15151515151515152
Precision: 0.19090909090909092
Recall: 0.16666666666666666
F1 Score: 0.15151515151515152


c:\Users\FPT\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
